## 함수형 API

* 기존의 시퀀셜 모델은 입력과 출력이 하나라고 가정함
* But, 일부 네트워크는 개별 **입력이 여러 개** 필요하거나 **출력이 여러 개** 필요
* 층이 차례대로 쌓이지 않고 층 사이가 연결된 **그래프 구조**도 있음


* n개의 다른 모델을 별도로 트레이닝 해서 각 예측을 가중 평균(Weighted average)해야 하는 상황
* 같은 피쳐에 대해 서로 다른 출력(장르, 시대 예측)을 동시에 내야 하는 경우 + 서로의 결과를 아는 것이 각각의 결과를 내는데에 도움이 될 때


* 최근의 뉴럴넷은 **선형적이지 않은** 네트워크 토폴로지가 필요
* **비순환 유향** 그래프 같은 구조 ex) 인셉션
* 잔차 연결(**residual**) 추가하는 경향 ex) ResNet
    - 하위 층의 출력 텐서를 상위 층의 출력 텐서에 더해서 아래층의 표현이 위쪽으로 흐를 수 있도록 하는 기법
    - 하위 층의 학습 결과가 데이터 처리 과정에서 손실되는 것을 방지


* 다중 입력, 다중 출력, 그래프 구조를 띈 모델을 만드는 법

In [1]:
from keras import Input, layers

input_tensor = Input(shape=(32,)) # 텐서
dense = layers.Dense(32, activation='relu') # 레이어 객체 (함수처럼 사용 가능)

output_tensor = dense(input_tensor) # 텐서와 함께 레이어 호출시 텐서 반환

Using TensorFlow backend.


## 기존의 Sequential Model

In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

## Functional API Model
- input_tensor, output_tensor 정보를 알려주면 케라스가 중간에 연결된 층을 모두 추출하고
- 모아 그래프 구조인 Model 객체 생성

In [3]:
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


## 이후의 컴파일, 트레이닝, 평가 과정은 같다

In [4]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 0s 355us/step - loss: 11.5820
Epoch 2/10
1000/1000 [==============================] - 0s 17us/step - loss: 11.5348
Epoch 3/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.5272
Epoch 4/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.5221
Epoch 5/10
1000/1000 [==============================] - 0s 18us/step - loss: 11.5185
Epoch 6/10
1000/1000 [==============================] - 0s 20us/step - loss: 11.5141
Epoch 7/10
1000/1000 [==============================] - 0s 20us/step - loss: 11.5118
Epoch 8/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.5083
Epoch 9/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.5058
Epoch 10/10
1000/1000 [==============================] - 0s 62us/step


## 다중 입력 모델
- 텐서를 더하거나 이어 붙이는 식으로 서로 다른 입력을 합치는 층을 만들 수 있다
    - keras.layers.add
    - keras.layers.concatenate
    - keras.layers.merge
        - .average(), .maximum(), .minimum(), .multiply(), .subtract(), .dot()
        
        
Question Answering model Example
- Input: Question, Context => Output: Answer

In [5]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), 
                   dtype='int32', 
                   name='text') # 길이가 정해지지 않은 정수 시퀀스
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), 
                       dtype='int32', 
                       name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question],
                                 axis=-1)

answer = layers.Dense(answer_vocabulary_size,
                     activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])

학습

In [6]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
answers = to_categorical(answers) # one-hot 인코딩 벡터

# 1
#model.fit([text, question], answers, epochs=10, batch_size=128)

# 2, dict 입력, name 지정시에만 가능
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.2145 - acc: 0.0040
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1982 - acc: 0.0340
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1447 - acc: 0.0090
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0642 - acc: 0.0070
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0148 - acc: 0.0080
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.9582 - acc: 0.0170
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.8909 - acc: 0.0130
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.8117 - acc: 0.0190
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.7367 - acc: 0.0190
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.6580 - acc: 0.0220


## 다중 출력 모델

사용자 포스트 입력 받아 나이, 성별, 소득 수준 예측

In [10]:
from keras import layers
from keras import Input
from keras.models import Model
vocabulary_size = 50000
num_incom_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_incom_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

트레이닝
- Gradient Descent는 하나의 스칼라 값을 최소화하기 때문에 이 모델을 트레이닝 시키기 위해선
    - 손실들을 하나의 값으로 합쳐야 한다.
    - but, 손실값들이 불균형할 경우 Loss가 가장 큰 값에 치우쳐 표현들을 최적화하기 때문에, 다른 작업들이 손해를 볼 수
    - 따라서 Loss 값이 최종 Loss에 기여하는 수준을 정할 수도 있다.
        - 특히 스케일이 다를 때, 예를들어 mse는 일반적으로 3~5사이의 값을 가진다면, 크로스엔트로피 loss는 0.1 정도로 낮음
        - 이런 경우 전자에 가중치 0.25, 후자에 가중치 10을 주는 식으로 구성할 수 있다.

In [14]:
model.compile(optimizer='rmsprop',
             loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

# 출력층에 이름 지정한 경우에만 가능
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                  'income': 'categorical_crossentropy',
                  'gender': 'binary_crossentropy'})

model.compile(optimizer='rmsprop',
             loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
             loss_weights=[0.25, 1., 10.])

model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                  'income': 'categorical_crossentropy',
                  'gender': 'binary_crossentropy'},
             loss_weights={'age': 0.25,
                          'income': 1.,
                          'gender': 10.})

fit은 다음과 같이
```py
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

model.fit(posts, {'age': age_targets,
                 'income': income_targets,
                 'gender': gender_Targets},
         epochs=10, batch_size=64)
```

## Directed Asyclic Graph
- 함수형 API를 통해 토폴로지가 복잡한 네트워크도 만들 수 있다.
    - 인셉션 모듈(Inception)
        - Inception https://arxiv.org/abs/1409.4842
        - Network in Network https://arxiv.org/abs/1312.4400
        - Inception V2, V3, https://arxiv.org/abs/1512.00567
        - Inception V4, https://arxiv.org/abs/1602.07261
        - keras.applications.inception_v3.InceptionV3에 네트워크 구조 및 사전 훈련 가중치 포함
        - 엑셉션(Xception)
            - https://arxiv.org/abs/1610.02357
            - **깊이별 합성곱 적용(depthwise conv** ; 각 입력 채널에 각각 적용되는 공간 방향 합성곱)
            - 인셉션 모듈의 극한 형태로 공간 특성과 채널 방향 특성 완전 분리
            - layers.SeparableConv2D 합성곱 사용 (채널에 각각 3x3 수행, 채널을 늘리지 않는 연산)
            - 그 다음 1x1 conv 적용함으로써 채널 수 늘림
            - keras.applications.xception.Xception
            - 인셉션 보다 더 빠르고 정확도가 높음(?)
    - 잔차 연결(Residual Connection)

![inception](./inception.png)

* **1x1 conv** (Network in Network논문에서 소개)
    - conv는 입력 텐서에서 타일 주변의 패치를 추출하고 각 패치에 동일한 연산을 수행
    - **1x1 conv(pointwise convolution)**는 패치가 하나의 타일 인 케이스
        - 이 conv연산은 모든 타일 벡터를 하나의 Dense층에 통과시키는 것과 동일
        - 즉, **입력 텐서의 채널 정보를 혼합한 특성 계산**으로 공간 방향으로는 정보를 섞지 않는다 (한번에 하나의 타일만 고려)
        - **채널 방향의 특성 학습과 공간 방향의 특성 학습을 분리**하는데 도움을 준다.
        
인셉션 모듈은 대략 다음과 같이 구현된다  
(실제로는 padding='same' 옵션으로 출력 크기를 맞춤)

```py
from keras import layers

# x : 4D tensor

branch_a =  layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(X)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layer.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)
```

* **Residual Connection**
    - **Vanishing Gradient**와
        - 트레이닝 알고리즘의 핵심인 역전파는 출력 손실에서 얻은 피드백을 하위층에 전파한다.
        - 이 때 신호가 **깊이 쌓인 층을 통해 전파되면 신호가 사라지거나 사라질 수** 있다.
            - LSTM은 별도의 트랙을 나란히 두어 중요정보를 유지함으로써 해결하였고
            - Residual Connection에선 특정 레이어에 나란히 단순한 선형 정보를 실어 나름으로써 해결
                - 하위 층의 출력과 상위 층의 출력을 그대로 더했으므로, 그래디언트가 그대로 잔차 연결을 따라 하위층으로 전달된다.
                - 잔차 연결을 따라 내려온 그래디언트는 메인 네트워크 층을 거쳐 줄어든 그래디언트와 더해진다.
    - **Representational bottleneck** 해결
        - Sequential 모델에서 표현 학습을 위한 층은 다른 층 위에 놓인다.
            - 어떤 층은 이전 층의 활성화 출력 정보만 사용
            - 이전 층이 너무 저차원이면 다음 층이 표현할 수 있는 정도가 제한될 수 (모델 성능 좌우)
        - 따라서 잔차 연결을 통해 하위 층의 정보를 다시 주입함으로써 손실된 정보를 복원할 수 있게 해줌
    - 일반적으로 10개 층 이상을 가진 모델에 잔차 연결을 추가하면 도움이 된다
    - **하위 층의 출력을 상위 층의 입력**으로 사용하는 기법

특성 맵의 크기가 같을 때 원본을 그대로 사용하는 Residual Connection 예시

```py
x = ...4D tensor
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x]) # 원본 x를 출력 특성에 더함 (두 출력의 크기가 동일해야 한다)
```

특성 맵의 크기가 다를 때 선형 변환을 사용해서 Residual Connection을 구현한 예

```py
x = ...4D tensor
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x) # y와 크기 맞추기 위해 1x1 conv적용 => 원본 x 다운샘플링

y = layers.add([y, residual])
```

## 층 가중치 공유
- 함수형 API를 통해, **레이어 객체를 여러번 재사용** 가능
- 레이어 두번 호출 시 각 호출에 동일한 가중치 재사용
- **표현을 공유**하고, **다른 입력에서 함께 학습**
    - ex) 2 문장의 유사도 판별 모델
        - 각 문장을 인코딩 하는 레이어가 서로 다를 필요가 없다
        - 두 문장의 순서가 바뀔 수도 있고, 의미가 비슷하다는 것은 대칭 관계를 의미
        - 독립된 2개의 레이어를 학습하는 대신, 하나의 LSTM 층으로 양쪽 모두 처리
        - **Siamese LSTM** 혹은 **공유 LSTM**이라 부름

In [15]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128)) # 128 차원의 가변길이 시퀀스
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
# model.fit([left_data, right_data], targets) # 양쪽 입력을 바탕으로 가중치 업데이트

## 층과 모델
- 함수형 API에서는 모델을 층처럼 사용 가능 (마치 커다란 층)
    - ex) y = model(x)
    - ex) y1, y2 = model([x1, x2]) # 입력 텐서 및 출력 텐서가 여러개 인경우
- 모델 객체를 호출할 경우 가중치가 재사용됨
    - ex) 깊이를 감지하기 위한, 듀얼 카메라 입력 (사진 2장) 

In [16]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None, include_top=False) # 최상단의 분류기 제외(전역 평균 풀링 및 FC Layer)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)